In [1]:
import pandas as pd
import numpy as np
from polyline.codec import PolylineCodec
from geopy import distance
from datetime import datetime

In [2]:
df = pd.read_csv('./final_data/train.csv')

In [3]:
df.head(5)

,Id,main_id_locality,ETA,RTA,OrderedDate,latitude,del_latitude,longitude,del_longitude,EDA,...,GoodArrived,ready_latitude,ready_longitude,onway_latitude,onway_longitude,arrived_latitude,arrived_longitude,center_latitude,center_longitude,route
0,0,1078,226.0,188.0,2020-02-12 19:12:06,55.826019,55.825581,49.134529,49.126949,1.0,...,2020-02-12 19:18:14,55.825647,49.134115,55.826261,49.134137,55.825706,49.127136,55.794388,49.111531,gnvsIaq{jHChA??uC???OPG^F^NRzKBd@AN[r@???`@`@`...
1,1,1078,718.0,725.0,2020-02-12 19:12:22,55.795502,55.820911,49.131470,49.115360,5.0,...,2020-02-12 19:31:47,55.795591,49.132281,55.795544,49.131552,55.821531,49.115685,55.794388,49.111531,sqpsI}~zjHyAr]e@lMk@fLaBlb@i@rLKhBCdAUxEGlCg@f...
2,2,1078,612.0,764.0,2020-02-12 19:12:44,55.791050,55.819962,49.226070,49.176628,5.0,...,2020-02-12 19:28:09,55.791099,49.226066,55.791099,49.226066,55.819765,49.177432,55.794388,49.111531,auosI}mmkH?LHd@KhC??o@w@[g@m@iAUk@??{G|OiB`Ek@...
3,3,1078,1560.0,1412.0,2020-02-12 19:12:44,55.753899,55.824680,49.188519,49.093700,13.0,...,2020-02-12 19:42:41,55.754116,49.188853,55.754116,49.188853,55.824734,49.094013,55.794388,49.111531,{lhsIiffkHmKN_C?mIPwMJ??Si@gA{B??Wq@MRCJTp@hAd...
4,4,1078,1528.0,893.0,2020-02-12 19:12:45,55.822361,55.786758,49.069092,49.143501,9.0,...,2020-02-12 19:32:19,55.822483,49.069659,55.822617,49.069229,55.786936,49.143330,55.794388,49.111531,yxusI{xnjHgAfG??}IuHkAqA??pIoe@VsA??dAkG`BuH??...


# 1. RTA based on driver data

In [4]:
def calc_rta(good_arrived, client_collected):
    ga_dt = datetime.strptime(good_arrived, '%Y-%m-%d %H:%M:%S')
    cc_dt = datetime.strptime(client_collected, '%Y-%m-%d %H:%M:%S')
    assert ga_dt >= cc_dt
    dif = ga_dt - cc_dt
    #assert dif.days == 0 # ( for taxi?)
    if dif.days != 0:
        print(good_arrived, client_collected)
    dif_in_seconds = dif.seconds
    return float(dif_in_seconds)

In [5]:
df['driver_rta'] = df.apply(lambda row: calc_rta(row['GoodArrived'], row['ClientCollected']), axis=1)

2020-02-05 04:28:58 2020-02-03 15:27:05
2020-02-26 10:20:05 2020-02-25 08:15:22
2020-02-11 08:51:50 2020-02-09 21:06:28
2020-02-09 04:47:02 2020-02-08 02:24:12


In [6]:
df.head(5)

,Id,main_id_locality,ETA,RTA,OrderedDate,latitude,del_latitude,longitude,del_longitude,EDA,...,ready_latitude,ready_longitude,onway_latitude,onway_longitude,arrived_latitude,arrived_longitude,center_latitude,center_longitude,route,driver_rta
0,0,1078,226.0,188.0,2020-02-12 19:12:06,55.826019,55.825581,49.134529,49.126949,1.0,...,55.825647,49.134115,55.826261,49.134137,55.825706,49.127136,55.794388,49.111531,gnvsIaq{jHChA??uC???OPG^F^NRzKBd@AN[r@???`@`@`...,188.0
1,1,1078,718.0,725.0,2020-02-12 19:12:22,55.795502,55.820911,49.131470,49.115360,5.0,...,55.795591,49.132281,55.795544,49.131552,55.821531,49.115685,55.794388,49.111531,sqpsI}~zjHyAr]e@lMk@fLaBlb@i@rLKhBCdAUxEGlCg@f...,725.0
2,2,1078,612.0,764.0,2020-02-12 19:12:44,55.791050,55.819962,49.226070,49.176628,5.0,...,55.791099,49.226066,55.791099,49.226066,55.819765,49.177432,55.794388,49.111531,auosI}mmkH?LHd@KhC??o@w@[g@m@iAUk@??{G|OiB`Ek@...,764.0
3,3,1078,1560.0,1412.0,2020-02-12 19:12:44,55.753899,55.824680,49.188519,49.093700,13.0,...,55.754116,49.188853,55.754116,49.188853,55.824734,49.094013,55.794388,49.111531,{lhsIiffkHmKN_C?mIPwMJ??Si@gA{B??Wq@MRCJTp@hAd...,1412.0
4,4,1078,1528.0,893.0,2020-02-12 19:12:45,55.822361,55.786758,49.069092,49.143501,9.0,...,55.822483,49.069659,55.822617,49.069229,55.786936,49.143330,55.794388,49.111531,yxusI{xnjHgAfG??}IuHkAqA??pIoe@VsA??dAkG`BuH??...,893.0


# 2. Initial metric value (ETA, driver_rta)

In [7]:
from sklearn.metrics import mean_squared_error

# for example, rmse

def rta_metric(eta, rta):
    return np.sqrt(mean_squared_error(eta, rta))

In [8]:
INITIAL_METRIC = rta_metric(df['ETA'], df['driver_rta'])

In [9]:
INITIAL_SIZE = df.shape[0]

# 3. Missing value columns

In [10]:
df.isnull().any()

Id                    False
main_id_locality      False
ETA                   False
RTA                   False
OrderedDate           False
latitude              False
del_latitude          False
longitude             False
del_longitude         False
EDA                   False
RDA                   False
ReadyForCollection    False
ClientCollected       False
GoodArrived           False
ready_latitude         True
ready_longitude        True
onway_latitude         True
onway_longitude        True
arrived_latitude       True
arrived_longitude      True
center_latitude       False
center_longitude      False
route                  True
driver_rta            False
dtype: bool

There are some missing values

# 4. Order incorrectness

In [11]:
def driver_data_incorrectness(eta, driver_rta,
                             latitude, longitude,
                             del_latitude, del_longitude,
                             onway_latitude, onway_longitude,
                             arrived_latitude, arrived_longitude):
    """
    some heuristics for incorrect data filled in by a driver
    """
    time_dif = np.square(eta - driver_rta)
    start = latitude, longitude
    end = del_latitude, del_longitude
    try:
        onway = onway_latitude, onway_longitude
        arrived = arrived_latitude, arrived_longitude
        spatial_start_dif = distance.great_circle(start, onway).m
        spatial_end_dif = distance.great_circle(end, arrived).m
        return time_dif + spatial_start_dif + spatial_end_dif
    except:
        return None

In [12]:
df['incorrectness'] = df.apply(lambda row: driver_data_incorrectness(row['ETA'], row['driver_rta'],
                                                   row['latitude'], row['longitude'],
                                                   row['del_latitude'], row['del_longitude'],
                                                   row['onway_latitude'], row['onway_longitude'],
                                                   row['arrived_latitude'], row['arrived_longitude']), axis=1)

In [13]:
df.head(5)

,Id,main_id_locality,ETA,RTA,OrderedDate,latitude,del_latitude,longitude,del_longitude,EDA,...,ready_longitude,onway_latitude,onway_longitude,arrived_latitude,arrived_longitude,center_latitude,center_longitude,route,driver_rta,incorrectness
0,0,1078,226.0,188.0,2020-02-12 19:12:06,55.826019,55.825581,49.134529,49.126949,1.0,...,49.134115,55.826261,49.134137,55.825706,49.127136,55.794388,49.111531,gnvsIaq{jHChA??uC???OPG^F^NRzKBd@AN[r@???`@`@`...,188.0,1498.553270
1,1,1078,718.0,725.0,2020-02-12 19:12:22,55.795502,55.820911,49.131470,49.115360,5.0,...,49.132281,55.795544,49.131552,55.821531,49.115685,55.794388,49.111531,sqpsI}~zjHyAr]e@lMk@fLaBlb@i@rLKhBCdAUxEGlCg@f...,725.0,127.748555
2,2,1078,612.0,764.0,2020-02-12 19:12:44,55.791050,55.819962,49.226070,49.176628,5.0,...,49.226066,55.791099,49.226066,55.819765,49.177432,55.794388,49.111531,auosI}mmkH?LHd@KhC??o@w@[g@m@iAUk@??{G|OiB`Ek@...,764.0,23164.207067
3,3,1078,1560.0,1412.0,2020-02-12 19:12:44,55.753899,55.824680,49.188519,49.093700,13.0,...,49.188853,55.754116,49.188853,55.824734,49.094013,55.794388,49.111531,{lhsIiffkHmKN_C?mIPwMJ??Si@gA{B??Wq@MRCJTp@hAd...,1412.0,21956.414881
4,4,1078,1528.0,893.0,2020-02-12 19:12:45,55.822361,55.786758,49.069092,49.143501,9.0,...,49.069659,55.822617,49.069229,55.786936,49.143330,55.794388,49.111531,yxusI{xnjHgAfG??}IuHkAqA??pIoe@VsA??dAkG`BuH??...,893.0,403277.194465


In [14]:
df.incorrectness.mean()

81884.50064312841

In [15]:
threshold = df.incorrectness.mean() * 1.5
selector = (df['incorrectness'] <= threshold)
df = df[selector]

In [16]:
CUR_SIZE = df.shape[0]
CUR_METRIC = rta_metric(df['ETA'], df['driver_rta'])

# Conclusion

In [17]:
CUR_SIZE / INITIAL_SIZE # >= 80%

0.8209117612445029

In [18]:
INITIAL_METRIC, CUR_METRIC

(304.81025323582656, 169.15756537678106)